In [35]:
import numpy as np
X = np.arange(0,10)
y = np.ones(len(X))
musk_y = [3,4,5,9]
y[musk_y] = -1
print(y)
def weak_classifier(m):
    y_predict = np.ones(len(X))
    for indx, x in enumerate(X):
        if(x > m):
            y_predict[indx] = -1
    return y_predict

def get_err_rate(y, y_pred):
    return 1 - len(y[y != y_predict])/len(y)

def get_coef_wc(err_rate):
    return 0.5 * np.log(1/err_rate - 1)

def get_normal_factor(Weights,cofe_wc, y, y_pred):
    return np.dot(Weights,np.exp(-coef_wc * y * y_pred))

def update_w(Weights,normal_factor,cofe_wc, y, y_pred):
    w = Weights
    for i in range(len(X)):
        w[i] = w[i]/ normal_factor * np.exp(-coef_wc*y[i]*y_pred[i])
    return w
W = np.ones(len(X))/len(X)
y_predict = weak_classifier(2.5)
coef_wc = get_coef_wc(0.3)
normal_factor = get_normal_factor(W, coef_wc,y, y_predict)
print(normal_factor)
print(update_w(W, normal_factor, coef_wc,y, y_predict))




[ 1.  1.  1. -1. -1. -1.  1.  1.  1. -1.]
0.916515138991
[ 0.07142857  0.07142857  0.07142857  0.07142857  0.07142857  0.07142857
  0.16666667  0.16666667  0.16666667  0.07142857]


In [60]:
def loadSimpData():
    # pred [-1, 1, -1,-1,1]   [1.0, 1.0, -1.0, -1.0, 1.0]
    datMat = np.matrix([[ 1. ,  2.1],   
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

def loadData():
    X = np.arange(0,10)
    y = np.ones(len(X))
    musk_y = [3,4,5,9]
    y[musk_y] = -1
    return X,y

datMat,classLabels = loadSimpData()
#datMat,classLabels = loadData()

print(datMat.shape)

(5, 2)


In [116]:

def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):#just classify the data
    retArray = np.ones((np.shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    #print("retArray",retArray)
    return retArray

def buildStump(dataArr,classLabels,D):
    
    dataMatrix = np.mat(dataArr)
    #print("shape",dataMatrix.shape)
    labelMat = np.mat(classLabels).T
    #m = 5, n = 2
    m,n = np.shape(dataMatrix)
    numSteps = 10.0 #用于在特征的所有可能值上进行遍历
    bestStump = {} #存储给定权重向量 D 时所得到的最佳单层决策树的信息
    bestClassEst = np.mat(np.zeros((m, 1)))
    minError = np.inf
    #for i in [0]:
    for i in range(n):
        rangeMin = dataMatrix[:, i].min()
        rangeMax = dataMatrix[:, i].max()
        stepSize = ( rangeMax - rangeMin) / numSteps
        #print("rangeMin",rangeMin,"rangeMax",rangeMax,"stepSize",stepSize)
        
        for j in range(-1, int(numSteps) + 1):
            for inequal in ['lt', 'gt']:
                threshVal = rangeMin + float(j) * stepSize
                #print(threshVal)
                predictedVals = stumpClassify(dataMatrix, i, threshVal, inequal )
                #print("predictedVals", predictedVals)
                errArr = np.mat(np.ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                #print("errArr", errArr)
                #errArr中分类错误的为1，weightedError为分类错误的点权重之和
                weightedError = (np.array(D) * np.array(errArr)).sum()
                #print("weightedError type",type(weightedError) )
                #print(inequal,"shreshVal",threshVal,"weightedError",weightedError)
                if weightedError < minError:
                    minError =  weightedError
                    bestClassEst = predictedVals.copy()
                    bestStump[ 'dim' ] = i
                    bestStump[ 'thresh' ] = threshVal
                    bestStump[ 'ineq' ] = inequal
                    
    return bestStump, minError, bestClassEst

D = np.array([0.2,0.2,0.2,0.2,0.2])
#D = np.ones(10)/10
#print(buildStump(datMat,classLabels,D))

def adaBoostTrain(dataArr, classLabels,numIt = 40):
    weakClfArr = []
    m = dataArr.shape[0]
    D = np.mat(np.ones((m,1)) / m)
    aggClassEst = np.mat(np.zeros((m,1)))
    
    for i in range(numIt):
        bestStump, error, classEst = buildStump(dataArr, classLabels, D )
        print(" D: ", D.T)
        #计算弱分类器的权重
        #print("error,type:",type(error))
        alpha = 0.5 * np.log((1/error - 1))
    
        bestStump[ 'alpha' ] = alpha
        weakClfArr.append(bestStump)
        #print("classEst: ", classEst.T)
        
        #更新D
        #print("classLabels",np.mat(classLabels).T, "classEst",classEst)
        #print("multi:",np.multiply(np.mat(classLabels).T , classEst))
        #print("alpha:",type(alpha))
        expon = np.multiply( -1 * alpha * np.mat( classLabels ).T, classEst)
        #print("expon:",expon)
        D = np.multiply(D, np.exp(expon))
        D = D / D.sum()
        
        # 记录对每个样本点的类别估计的累积值
        aggClassEst += alpha * classEst
        print("aggClassEst: ", aggClassEst.T)
        
        # 计算分类错误率
        classEst_ = np.array(classEst.T).reshape(m)
        classLabels_ = np.array(classLabels)
        print("shape:",classEst_ ,classLabels_)
        errorRate = len(classLabels_[classEst_ != classLabels_])/ m
        print("total error: ", errorRate)
        
        # 如果完全正确，终止迭代
        if errorRate == 0.0:
            break
    return weakClfArr

print(adaBoostTrain( datMat, classLabels, 9))
        
    


 D:  [[ 0.2  0.2  0.2  0.2  0.2]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
shape: [-1.  1. -1. -1.  1.] [ 1.  1. -1. -1.  1.]
total error:  0.2
 D:  [[ 0.5    0.125  0.125  0.125  0.125]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
shape: [ 1.  1. -1. -1. -1.] [ 1.  1. -1. -1.  1.]
total error:  0.2
 D:  [[ 0.28571429  0.07142857  0.07142857  0.07142857  0.5       ]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
shape: [ 1.  1.  1.  1.  1.] [ 1.  1. -1. -1.  1.]
total error:  0.4
 D:  [[ 0.16666667  0.04166667  0.25        0.25        0.29166667]]
aggClassEst:  [[ 0.37096867  3.36670095 -1.57494148 -1.57494148  1.4207908 ]]
shape: [-1.  1. -1. -1.  1.] [ 1.  1. -1. -1.  1.]
total error:  0.2
 D:  [[ 0.5    0.025  0.15   0.15   0.175]]
aggClassEst:  [[ 1.14626738  4.14199965 -2.35024018 -2.35024018  0.64549209]]
shape: [ 1.  1. -1. -1. -1.] [ 1.  1. -1. -1.  1.]
total error:  0.2
 D:  

In [186]:
#按统计学习方法重写


import numpy as np
X = np.arange(0,10)
y = np.ones(len(X))
musk_y = [3,4,5,9]
y[musk_y] = -1

m = len(X)

def weak_classify(datas, threshVal, threshIneq):#just classify the data
    retArray = np.ones(m)
    if threshIneq == 'lt':
        retArray[ datas <= threshVal] = -1.0
    else:
        retArray[datas > threshVal] = -1.0
    return retArray

def buildStump(datas,labels,D):
    steps = 10.0 #用于在特征的所有可能值上进行遍历
    best_stump = {} #存储给定权重向量 D 时所得到的最佳单层决策树的信息
    pred_y = np.zeros(m)
    min_err = np.inf
    
    x_min = datas.min()
    x_max = datas.max()
    step_len = (x_max - x_min + 1) / steps
    for i in range(-1, int(steps)):
        for inequal in ['lt', 'gt']:
            thresh = (x_min + 0.5) + i * step_len
            #print("threshVal:",thresh)
            y_pred = weak_classify(datas,thresh, inequal)
            #print("true",labels)
            #print("pred", y_pred)
            err_w = get_err_rate(labels, y_pred, D)
            #print("inequal:",inequal,"err_w:",err_w)
            if err_w < min_err:
                min_err =  err_w
                best_stump['min_err'] = min_err
                best_stump['y_pred_best'] = y_pred.copy()
                #bestStump[ 'dim' ] = i
                best_stump['alpha'] = get_alpha(min_err)
                best_stump[ 'thresh' ] = thresh
                best_stump[ 'ineq' ] = inequal
    return best_stump

def get_err_rate(y, y_pred,D):
    pred_result = np.array(y != y_pred,dtype = np.float32)
    #print("t or f:",y != y_pred)
    return np.dot(D,pred_result)

def get_err_index(y, y_pred):
    indx = []
    for i in range(len(y)):
        if(y[i] !=  y_pred[i]):
            indx.append(i)
    return indx
#     pred_result = np.array(y != y_pred,dtype = np.float32)
#     indx = np.where(pred_result == 1.0)
#     return indx

def get_alpha(err_rate):
    return 0.5 * np.log(1/err_rate - 1)

# def get_normal_factor(Weights,cofe_wc, y, y_pred):
#     return np.dot(Weights,np.exp(-coef_wc * y * y_pred))

def update_D(D,alpha, y, y_pred):
    #D_new.shape:(5,1)
    D_new = D * np.exp(-alpha * y * y_pred)
    D_new = D_new/D_new.sum()
    return D_new

def get_combine_cls(models):
    y_ = np.zeros(m)
    for model in models:
        y_ +=  model['alpha'] * model['y_pred_best']
    y_[y_ > 0] = 1.0
    y_[y_ < 0] = -1.0
    return y_

D = np.ones(len(X))/len(X)
print(D)

models = [] 
iter_num = 9
for i in range(iter_num):
    best_stump = buildStump(X,y,D)
    alpha = best_stump['alpha']
    y_pred = best_stump['y_pred_best']
    models.append(best_stump)
    D = update_D(D,alpha,y,y_pred)
    y_ = get_combine_cls(models)
    indx = get_err_index(y, y_)
    if indx:
        print("第",i+1,"次，合成分类器分类错误的数据下标为:",indx)
    else:
        print("第",i+1,"次全部分类正确，迭代结束")
        break
for model in models:
    print(model)


    
        


# err_rate_w = get_err_rate(y, y_predict,D)
# print(err_rate_w)
# coef_wc = get_coef_wc(0.3)
# normal_factor = get_normal_factor(W, coef_wc,y, y_predict)
# print(normal_factor)
# print(update_w(W, normal_factor, coef_wc,y, y_predict))

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
第 1 次，合成分类器分类错误的数据下标为: [6, 7, 8]
第 2 次，合成分类器分类错误的数据下标为: [3, 4, 5]
第 3 次全部分类正确，迭代结束
{'ineq': 'gt', 'min_err': 0.30000000000000004, 'y_pred_best': array([ 1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.]), 'thresh': 2.5, 'alpha': 0.42364893019360172}
{'ineq': 'gt', 'min_err': 0.21428571428571427, 'y_pred_best': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.]), 'thresh': 8.5, 'alpha': 0.64964149206513044}
{'ineq': 'lt', 'min_err': 0.18181818181818185, 'y_pred_best': array([-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.]), 'thresh': 5.5, 'alpha': 0.75203869838813697}
